In [14]:
import pandas as pd 
import os

In [15]:
csv_path = os.path.join('..', 'Inputs', 'StationPoints.CSV')

In [16]:
os.path.exists(csv_path)

True

In [17]:
df = pd.read_csv(csv_path)

In [18]:
column_mapping = {
        'NAME': 'Key',
        'DESC': 'Name',
        'ZONEID': 'pAORGroup',
        'ALRMPRIOR': 'Priority'
    }
df = df.rename(columns=column_mapping)

In [19]:
df['Priority']

0     0
1     0
2     0
3     0
4     0
     ..
62    0
63    0
64    0
65    0
66    3
Name: Priority, Length: 67, dtype: int64

In [20]:


def scada_processing():
    print("Hello World from scada_processing!")




if __name__ == "__main__":
    scada_processing()

Hello World from scada_processing!
